In [1]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Input, Activation, Dropout, Add, LSTM, GRU, RNN, LayerNormalization, BatchNormalization, Conv1D, MaxPooling1D, Flatten
from keras.optimizers import Adam,SGD
import tensorflow as tf
from keras import Model, regularizers, activations
import pickle
from copy import deepcopy
import random

# disable warnings to ignore overflow error
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [2]:
# parameters for CSTR
T_0_cstr = 300
V_cstr = 1
k_0_cstr = 8.46*(np.power(10,6))
C_p_cstr = 0.231
rho_L_cstr = 1000
Q_s_cstr = 0.0
F_cstr = 5
E_cstr = 5*(np.power(10,4))
delta_H_cstr = -1.15*(np.power(10,4))
R_cstr = 8.314
C_A0s_cstr = 4

t_final_cstr = 0.005
t_step_cstr = 1e-4

# parameters for Batch
V_batch = 1
k_0_batch = 8.46*(np.power(10,7))
C_p_batch = 0.231
rho_L_batch = 1000
Q_s_batch = 0
E_batch = 5*(np.power(10,4))
delta_H_batch = -1.15*(np.power(10,4))
R_batch = 8.314

t_final_batch = 0.05
t_step_batch = 1e-4

# parameters for neural networks
num_step = 10
num_dims = 4

# parameters for Reptile
seed = 0
plot = True
rng = np.random.RandomState(seed)
innerstepsize = 0.02 # stepsize in inner SGD for reptile
innerepochs = 1 # number of epochs of each inner SGD
outerstepsize0 = 0.1 # stepsize of outer optimization, i.e., meta-optimization
niterations = 1000 # number of outer updates; each iteration we sample one task and update on it
epochs = 1 # number of epochs
ntrain = 32 # size of training minibatches (K)
eval_step = 50 # evaluation step
threshold = 10 # threshold to check data correctness

In [3]:
def generate_new(x):
    return x + x * np.random.uniform(-10, 10)

def generate_new_small(x):
    return x + x * np.random.uniform(-0.05, 0.05)

def Batch_simulation(V, k_0, E, R, delta_H, rho_L, C_p, Q, t_final, t_step, C_A_initial, T_initial):
    """
        simulating Batch using forward Euler method
    """

    C_A_list = list()  # evolution of CA over time
    T_list = list()  # evolution of T over time

    C_A = C_A_initial
    T = T_initial

    for i in range(int(t_final / t_step)):
        dCAdt = -k_0 * np.exp(-E / (R * T)) * C_A
        dTdt = - delta_H / (rho_L * C_p) * k_0 * np.exp(-E / (R * T)) * C_A + Q / (rho_L * C_p * V)

        T += dTdt * t_step
        C_A += dCAdt * t_step

        if i % 50 == 0:
            C_A_list.append(C_A)
            T_list.append(T)

    return C_A_list, T_list

def CSTR_simulation(F, V, C_A0, k_0, E, R, T_0, delta_H, rho_L, C_p, Q, t_final, t_step, C_A_initial, T_initial):
    """
        simulating CSTR using forward Euler method
    """

    C_A_list = list()  # evolution of CA over time
    T_list = list()  # evolution of T over time

    C_A = C_A_initial
    T = T_initial

    for i in range(int(t_final / t_step)):
        dCAdt = F / V * (C_A0 - C_A) - k_0 * np.exp(-E / (R * T)) * C_A**2
        dTdt = F / V * (T_0 - T) - delta_H / (rho_L * C_p) * k_0 * np.exp(-E / (R * T)) * C_A**2 + Q / (rho_L * C_p * V)

        T += dTdt * t_step
        C_A += dCAdt * t_step

        if i % 5 == 0:
            C_A_list.append(C_A)
            T_list.append(T)

    return C_A_list, T_list

def to_tensor(x):
    return tf.convert_to_tensor(x, dtype=tf.float32)

def train_on_batch(x, y, model, optimizer):
    x = to_tensor(x)
    y = to_tensor(y)

    with tf.GradientTape() as tape:
        YHat = model(x)
        loss = mse_loss_fn(y, YHat)
        grads = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    return loss

def predict(x, model):
    x = to_tensor(x)
    return model(x).numpy()

def compute_loss(x, y, model):
    return np.square(predict(x, model) - y).mean()

In [4]:
def gen_cstr(F, V, C_A0s, k_0, E, R, T_0, delta_H, rho_L, C_p, Q_s, t_final, t_step, num_step):
    isCorrect = False
    while isCorrect == False:
        T_0_new = generate_new(T_0)
        V_new = generate_new(V)
        F_new = generate_new(F)
        C_A0s_new = generate_new(C_A0s)
        Q_s_new = generate_new(Q_s)
        rho_L_new = generate_new_small(rho_L)
        C_p_new = generate_new_small(C_p)
        k_0_new = generate_new_small(k_0)
        E_new = generate_new_small(E)
        delta_H_new = generate_new_small(delta_H)

        # generating inputs and initial states for CSTR
        u1_list = np.linspace(-3.5, 3.5, 4, endpoint=True)
        u2_list = np.linspace(-5e5, 5e5, 4, endpoint=True)
        T_initial = np.linspace(300, 600, 20, endpoint=True)
        CA_initial = np.linspace(0, 6, 20, endpoint=True)

        # restruture the data
        T_start = list()
        CA_start = list()

        for T in T_initial:
            for CA in CA_initial:
                CA_start.append(CA)
                T_start.append(T)

        CA_start = np.array([CA_start])
        T_start = np.array([T_start])
        x_deviation = np.concatenate((CA_start.T, T_start.T), axis=1)

        # get X and y data for training and testing
        CA_output = list()
        T_output = list()
        CA_input = list()
        T_input = list()
        CA0_input = list()
        Q_input = list()

        for u1 in u1_list:
            C_A0 = u1 + C_A0s_new
            for u2 in u2_list:
                Q = u2 + Q_s_new
                for C_A_initial, T_initial in x_deviation:

                    C_A_list, T_list = CSTR_simulation(F_new, V_new, C_A0, k_0_new, E_new, R, T_0_new, delta_H_new, rho_L_new, C_p_new, Q, t_final, t_step, C_A_initial, T_initial)
                    if any(abs(i) < 0.001 for i in T_list) == False and any(abs(i) < 0.001 for i in C_A_list) == False and any(abs(i) > 10000 for i in T_list) == False and any(abs(i) > 10000 for i in C_A_list) == False and any(abs(i) == 0 for i in T_list) == False and any(abs(i) == 0 for i in C_A_list) == False and np.isnan(C_A_list).any() == False and np.isnan(T_list).any() == False and np.isinf(C_A_list).any() == False and np.isinf(T_list).any() == False:
                        CA0_input.append(u1)
                        Q_input.append(u2)
                        CA_input.append(C_A_initial)
                        T_input.append(T_initial)

                        CA_output.append(C_A_list)
                        T_output.append(T_list)

        # regenerate data if requirement is not met
        if len(CA_output) > 5000:

            # collate input for RNN
            CA0_input = np.array(CA0_input)
            CA0_input = CA0_input.reshape(-1,1,1)

            Q_input = np.array(Q_input)
            Q_input = Q_input.reshape(-1,1,1)

            CA_input = np.array(CA_input)
            CA_input = CA_input.reshape(-1,1,1)

            T_input = np.array(T_input)
            T_input = T_input.reshape(-1,1,1)

            RNN_input = np.concatenate((T_input, CA_input, Q_input, CA0_input), axis=2)
            RNN_input = RNN_input.repeat(num_step, axis=1)

            # collate output for RNN
            CA_output = np.array(CA_output)
            CA_output = CA_output.reshape(-1, num_step, 1)

            T_output = np.array(T_output)
            T_output = T_output.reshape(-1, num_step, 1)

            RNN_output = np.concatenate((T_output, CA_output), axis=2)

            # scale the data
            scaler_X = preprocessing.StandardScaler().fit(RNN_input.reshape(-1, num_dims))
            scaler_y = preprocessing.StandardScaler().fit(RNN_output.reshape(-1, 2))

            X = scaler_X.transform(RNN_input.reshape(-1, num_dims))
            y = scaler_y.transform(RNN_output.reshape(-1,2))

            if np.isnan(X).any() == False and np.isnan(y).any() == False and np.isinf(X).any() == False and np.isinf(y).any() == False and any(abs(i) > threshold for i in y.reshape(-1)) == False:
                isCorrect = True

    print("Number of training samples of CSTR: ", int(len(X)/num_step))
    return X.reshape(-1,num_step,num_dims), y.reshape(-1,num_step,2)

In [5]:
def gen_batch(V, k_0, E, R, delta_H, rho_L, C_p, Q_s, t_final, t_step, num_step):
    isCorrect = False
    while isCorrect == False:
        V_new = generate_new(V)
        Q_s_new = generate_new(Q_s)
        rho_L_new = generate_new_small(rho_L)
        C_p_new = generate_new_small(C_p)
        k_0_new = generate_new(k_0)
        E_new = generate_new_small(E)
        delta_H_new = generate_new_small(delta_H)

        # generating inputs and initial states for Batch
        u_list = np.linspace(-5e5, 5e5, 4, endpoint=True)
        T_initial = np.linspace(300, 600, 40, endpoint=True)
        CA_initial = np.linspace(0, 6, 40, endpoint=True)

        # restruture the data
        T_start = list()
        CA_start = list()

        for T in T_initial:
            for CA in CA_initial:
                CA_start.append(CA)
                T_start.append(T)

        CA_start = np.array([CA_start])
        T_start = np.array([T_start])
        x_deviation = np.concatenate((CA_start.T, T_start.T), axis=1)

        # get X and y data for training and testing
        CA_output = list()
        T_output = list()
        CA_input = list()
        T_input = list()
        CA0_input = list()
        Q_input = list()

        for u1 in u_list:
            Q = u1 + Q_s_new
            for C_A_initial, T_initial in x_deviation:

                C_A_list, T_list = Batch_simulation(V_new, k_0_new, E_new, R, delta_H_new, rho_L_new, C_p_new, Q, t_final, t_step, C_A_initial, T_initial)
                if any(abs(i) < 0.001 for i in T_list) == False and any(abs(i) < 0.001 for i in C_A_list) == False and any(abs(i) > 10000 for i in T_list) == False and any(abs(i) > 10000 for i in C_A_list) == False and any(abs(i) == 0 for i in T_list) == False and any(abs(i) == 0 for i in C_A_list) == False and np.isnan(C_A_list).any() == False and np.isnan(T_list).any() == False and np.isinf(C_A_list).any() == False and np.isinf(T_list).any() == False:
                    CA0_input.append(0)
                    Q_input.append(u1)
                    CA_input.append(C_A_initial)
                    T_input.append(T_initial)

                    CA_output.append(C_A_list)
                    T_output.append(T_list)

        # regenerate data if requirement is not met
        if len(CA_output) > 5000:

            # collate input for RNN
            CA0_input = np.array(CA0_input)
            CA0_input = CA0_input.reshape(-1,1,1)

            Q_input = np.array(Q_input)
            Q_input = Q_input.reshape(-1,1,1)

            CA_input = np.array(CA_input)
            CA_input = CA_input.reshape(-1,1,1)

            T_input = np.array(T_input)
            T_input = T_input.reshape(-1,1,1)

            RNN_input = np.concatenate((T_input, CA_input, Q_input, Q_input), axis=2)
            RNN_input = RNN_input.repeat(num_step, axis=1)

            # collate output for RNN
            CA_output = np.array(CA_output)
            CA_output = CA_output.reshape(-1, num_step, 1)

            T_output = np.array(T_output)
            T_output = T_output.reshape(-1, num_step, 1)

            RNN_output = np.concatenate((T_output, CA_output), axis=2)

            # scale the data
            scaler_X = preprocessing.StandardScaler().fit(RNN_input.reshape(-1, num_dims))
            scaler_y = preprocessing.StandardScaler().fit(RNN_output.reshape(-1, 2))

            X = scaler_X.transform(RNN_input.reshape(-1, num_dims))
            y = scaler_y.transform(RNN_output.reshape(-1,2))

            if np.isnan(X).any() == False and np.isnan(y).any() == False and np.isinf(X).any() == False and np.isinf(y).any() == False and any(abs(i) > threshold for i in y.reshape(-1)) == False:
                isCorrect = True

    print("Number of training samples of Batch: ", int(len(X)/num_step))
    return X.reshape(-1,num_step,num_dims), y.reshape(-1,num_step,2)

In [6]:
class Model(tf.keras.layers.Layer):

    def __init__(self):
        super(Model, self).__init__()

        self.layer_1 = SimpleRNN(64, activation='relu', return_sequences=True)
        self.layer_2 = SimpleRNN(64, activation='relu', return_sequences=True)
        self.layer_3 = Dense(2, activation='linear')

    def call(self, inputs):
        x = self.layer_1(inputs)
        x = self.layer_2(x)
        x = self.layer_3(x)
        return x

model = Model()

# Necessary to create the model's state.
# The model doesn't have a state until it's called at least once.
_ = model(tf.zeros((ntrain, num_step, num_dims)))

optimizer = tf.keras.optimizers.Adam()
mse_loss_fn = tf.keras.losses.MeanSquaredError()

In [7]:
# generate testing cstr
isOverflow = True
while isOverflow == True:
    try:
        x_test_cstr, y_test_cstr = gen_cstr(F_cstr, V_cstr, C_A0s_cstr, k_0_cstr, E_cstr, R_cstr, T_0_cstr, delta_H_cstr, rho_L_cstr, C_p_cstr, Q_s_cstr, t_final_cstr, t_step_cstr, num_step)
        isOverflow = False
    except ValueError:
        pass

sample_idx = rng.choice(len(x_test_cstr), size=1)
xtest_plot_1shot_cstr = x_test_cstr[sample_idx]
ytest_plot_1shot_cstr = y_test_cstr[sample_idx]

sample_idx = rng.choice(len(x_test_cstr), size=5)
xtest_plot_5shot_cstr = x_test_cstr[sample_idx]
ytest_plot_5shot_cstr = y_test_cstr[sample_idx]

sample_idx = rng.choice(len(x_test_cstr), size=10)
xtest_plot_10shot_cstr = x_test_cstr[sample_idx]
ytest_plot_10shot_cstr = y_test_cstr[sample_idx]

# generate testing batch
isOverflow = True
while isOverflow == True:
    try:
        x_test_batch, y_test_batch = gen_batch(V_batch, k_0_batch, E_batch, R_batch, delta_H_batch, rho_L_batch, C_p_batch, Q_s_batch, t_final_batch, t_step_batch, num_step)
        isOverflow = False
    except ValueError:
        pass

sample_idx = rng.choice(len(x_test_batch), size=1)
xtest_plot_1shot_batch = x_test_batch[sample_idx]
ytest_plot_1shot_batch = y_test_batch[sample_idx]

sample_idx = rng.choice(len(x_test_batch), size=5)
xtest_plot_5shot_batch = x_test_batch[sample_idx]
ytest_plot_5shot_batch = y_test_batch[sample_idx]

sample_idx = rng.choice(len(x_test_batch), size=10)
xtest_plot_10shot_batch = x_test_batch[sample_idx]
ytest_plot_10shot_batch = y_test_batch[sample_idx]

Number of training samples of CSTR:  6365
Number of training samples of Batch:  6101


In [8]:
training_loss = np.zeros(niterations)
test_loss_1shot_cstr = np.zeros(int(niterations/eval_step+1))
test_loss_5shot_cstr = np.zeros(int(niterations/eval_step+1))
test_loss_10shot_cstr = np.zeros(int(niterations/eval_step+1))
test_loss_1shot_batch = np.zeros(int(niterations/eval_step+1))
test_loss_5shot_batch = np.zeros(int(niterations/eval_step+1))
test_loss_10shot_batch = np.zeros(int(niterations/eval_step+1))
count = 0
countBatch = 0
countCSTR = 0

reactor = 0

# reptile
for iteration in range(niterations):

    print("Begin iteration ", iteration)

    # begin reptile
    weights_before = deepcopy(model.get_weights())

    # generate task
    isOverflow = True
    while isOverflow == True:
        try:
            # switch to another reactor, 0 for cstr and 1 for batch
            # reactor = random.randint(0,1)
            if reactor == 0:
                x_all, y_all = gen_cstr(F_cstr, V_cstr, C_A0s_cstr, k_0_cstr, E_cstr, R_cstr, T_0_cstr, delta_H_cstr, rho_L_cstr, C_p_cstr, Q_s_cstr, t_final_cstr, t_step_cstr, num_step)
                countCSTR = countCSTR + 1
                reactor = 1
            elif reactor == 1:
                x_all, y_all = gen_batch(V_batch, k_0_batch, E_batch, R_batch, delta_H_batch, rho_L_batch, C_p_batch, Q_s_batch, t_final_batch, t_step_batch, num_step)
                countBatch = countBatch + 1
                reactor = 0
            isOverflow = False
        except ValueError:
            pass

    inds = rng.permutation(len(x_all))

    # for some k number of iterations perform optimization on the task
    for k in range(innerepochs):

        for start in range(0, len(x_all), ntrain):
            mbinds = inds[start:start+ntrain]
            train_on_batch(x_all[mbinds], y_all[mbinds], model, optimizer)

    weights_after = model.get_weights()

    outerstepsize = outerstepsize0 * (1 - iteration / niterations) # linear schedule

    model.set_weights([ weights_before[i] + (weights_after[i] - weights_before[i]) * outerstepsize for i in range(len(model.weights))])

    training_loss[iteration] = compute_loss(x_all, y_all, model)
    print("Training loss: ", training_loss[iteration])

    # reset weights and terminate for NaN training loss
    if np.isnan(training_loss[iteration]) == True:
        model.set_weights(weights_before)
        break

    # begin evaluation
    if plot and iteration==0 or (iteration+1) % eval_step == 0:
        # 1 shot learning for cstr
        weights_before = deepcopy(model.get_weights())
        for inneriter in range(50):
            train_on_batch(xtest_plot_1shot_cstr, ytest_plot_1shot_cstr, model, optimizer)

        test_loss_1shot_cstr[count] = compute_loss(x_test_cstr, y_test_cstr, model)
        print("Test loss for 1-shot CSTR: ", test_loss_1shot_cstr[count])
        model.set_weights(weights_before)

        # 5 shot learning for cstr
        weights_before = deepcopy(model.get_weights())
        for inneriter in range(50):
            train_on_batch(xtest_plot_5shot_cstr, ytest_plot_5shot_cstr, model, optimizer)

        test_loss_5shot_cstr[count] = compute_loss(x_test_cstr, y_test_cstr, model)
        print("Test loss for 5-shots CSTR: ", test_loss_5shot_cstr[count])
        model.set_weights(weights_before)

        # 10 shot learning for cstr
        weights_before = deepcopy(model.get_weights())
        for inneriter in range(50):
            train_on_batch(xtest_plot_10shot_cstr, ytest_plot_10shot_cstr, model, optimizer)

        test_loss_10shot_cstr[count] = compute_loss(x_test_cstr, y_test_cstr, model)
        print("Test loss for 10-shots CSTR: ", test_loss_10shot_cstr[count])
        model.set_weights(weights_before)

        # 1 shot learning for batch
        weights_before = deepcopy(model.get_weights())
        for inneriter in range(50):
            train_on_batch(xtest_plot_1shot_batch, ytest_plot_1shot_batch, model, optimizer)

        test_loss_1shot_batch[count] = compute_loss(x_test_batch, y_test_batch, model)
        print("Test loss for 1-shot Batch: ", test_loss_1shot_batch[count])
        model.set_weights(weights_before)

        # 5 shot learning for bacth
        weights_before = deepcopy(model.get_weights())
        for inneriter in range(50):
            train_on_batch(xtest_plot_5shot_batch, ytest_plot_5shot_batch, model, optimizer)

        test_loss_5shot_batch[count] = compute_loss(x_test_batch, y_test_batch, model)
        print("Test loss for 5-shots Batch: ", test_loss_5shot_batch[count])
        model.set_weights(weights_before)

        # 10 shot learning for batch
        weights_before = deepcopy(model.get_weights())
        for inneriter in range(50):
            train_on_batch(xtest_plot_10shot_batch, ytest_plot_10shot_batch, model, optimizer)

        test_loss_10shot_batch[count] = compute_loss(x_test_batch, y_test_batch, model)
        print("Test loss for 10-shots Batch: ", test_loss_10shot_batch[count])
        model.set_weights(weights_before)

        count = count + 1

    if (iteration+1) % 100 == 0:
        filename = 'model_reptile_cstr_batch.sav'
        pickle.dump(model, open(filename, 'wb'))

Begin iteration  0
Number of training samples of CSTR:  6360
Training loss:  0.9596600574645293
Test loss for 1-shot CSTR:  0.7973811934127902
Test loss for 5-shots CSTR:  0.7334650338987341
Test loss for 10-shots CSTR:  0.558167584185795
Test loss for 1-shot Batch:  0.9641358538950744
Test loss for 5-shots Batch:  0.8736279297746337
Test loss for 10-shots Batch:  0.613496678550573
Begin iteration  1
Number of training samples of Batch:  6240
Training loss:  0.8734645986066967
Begin iteration  2
Number of training samples of CSTR:  6364
Training loss:  0.7743253351785527
Begin iteration  3
Number of training samples of Batch:  6240
Training loss:  0.7025788128051212
Begin iteration  4
Number of training samples of CSTR:  6400
Training loss:  0.6578732216249784
Begin iteration  5
Number of training samples of Batch:  6012
Training loss:  0.7865977517154898
Begin iteration  6
Number of training samples of CSTR:  6400
Training loss:  0.6735818261670375
Begin iteration  7
Number of trainin

Number of training samples of CSTR:  6369
Training loss:  0.46042007337567176
Begin iteration  79
Number of training samples of Batch:  6240
Training loss:  0.40917673404510124
Begin iteration  80
Number of training samples of CSTR:  6350
Training loss:  0.39154218742537644
Begin iteration  81
Number of training samples of Batch:  6044
Training loss:  0.603308415123441
Begin iteration  82
Number of training samples of CSTR:  6400
Training loss:  0.45775239900357195
Begin iteration  83
Number of training samples of Batch:  5753
Training loss:  0.6381374784619624
Begin iteration  84
Number of training samples of CSTR:  6240
Training loss:  0.47238057779360637
Begin iteration  85
Number of training samples of Batch:  6210
Training loss:  0.44698441768736236
Begin iteration  86
Number of training samples of CSTR:  6080
Training loss:  0.5799640527511775
Begin iteration  87
Number of training samples of Batch:  6240
Training loss:  0.31575361230818444
Begin iteration  88
Number of training 

Training loss:  0.36073156614578294
Begin iteration  156
Number of training samples of CSTR:  6384
Training loss:  0.45366988627322735
Begin iteration  157
Number of training samples of Batch:  6131
Training loss:  0.6173830221570057
Begin iteration  158
Number of training samples of CSTR:  6080
Training loss:  0.41046111399715896
Begin iteration  159
Number of training samples of Batch:  6240
Training loss:  0.35369467966602974
Begin iteration  160
Number of training samples of CSTR:  6359
Training loss:  0.27039426975785924
Begin iteration  161
Number of training samples of Batch:  6240
Training loss:  0.3686414801903334
Begin iteration  162
Number of training samples of CSTR:  6400
Training loss:  0.3768449498888231
Begin iteration  163
Number of training samples of Batch:  6240
Training loss:  0.4927639294241746
Begin iteration  164
Number of training samples of CSTR:  6080
Training loss:  0.3946480548983686
Begin iteration  165
Number of training samples of Batch:  6240
Training l

Training loss:  0.35734129983793905
Begin iteration  236
Number of training samples of CSTR:  6064
Training loss:  0.3733145825808098
Begin iteration  237
Number of training samples of Batch:  6228
Training loss:  0.549116590314804
Begin iteration  238
Number of training samples of CSTR:  6400
Training loss:  0.4514863924807023
Begin iteration  239
Number of training samples of Batch:  6240
Training loss:  0.2636724591053747
Begin iteration  240
Number of training samples of CSTR:  6400
Training loss:  0.36378144818499425
Begin iteration  241
Number of training samples of Batch:  6206
Training loss:  0.5922895379997457
Begin iteration  242
Number of training samples of CSTR:  6400
Training loss:  0.2915510860531385
Begin iteration  243
Number of training samples of Batch:  6162
Training loss:  0.6555318687073655
Begin iteration  244
Number of training samples of CSTR:  6400
Training loss:  0.3427476045705915
Begin iteration  245
Number of training samples of Batch:  6240
Training loss:

Training loss:  0.3779706223403508
Begin iteration  313
Number of training samples of Batch:  6240
Training loss:  0.3303387457523562
Begin iteration  314
Number of training samples of CSTR:  6400
Training loss:  0.29273510604569325
Begin iteration  315
Number of training samples of Batch:  6185
Training loss:  0.501650440602279
Begin iteration  316
Number of training samples of CSTR:  6300
Training loss:  0.32052519644214716
Begin iteration  317
Number of training samples of Batch:  6240
Training loss:  0.34024056818115406
Begin iteration  318
Number of training samples of CSTR:  6365
Training loss:  0.3138216036252024
Begin iteration  319
Number of training samples of Batch:  6240
Training loss:  0.3498910081812338
Begin iteration  320
Number of training samples of CSTR:  6400
Training loss:  0.31101986330892306
Begin iteration  321
Number of training samples of Batch:  6240
Training loss:  0.37776626347160686
Begin iteration  322
Number of training samples of CSTR:  6400
Training lo

...

Training loss:  0.3180827575828301
Begin iteration  627
Number of training samples of Batch:  6213
Training loss:  0.4915463237047363
Begin iteration  628
Number of training samples of CSTR:  6384
Training loss:  0.36968563602004323
Begin iteration  629
Number of training samples of Batch:  6240
Training loss:  0.283931500233531
Begin iteration  630
Number of training samples of CSTR:  6400
Training loss:  0.34201667302335964
Begin iteration  631
Number of training samples of Batch:  6240
Training loss:  0.4525037122037178
Begin iteration  632
Number of training samples of CSTR:  6288
Training loss:  0.2926830087565119
Begin iteration  633
Number of training samples of Batch:  6231
Training loss:  0.4357414168820132
Begin iteration  634
Number of training samples of CSTR:  6366
Training loss:  0.24541791503866672
Begin iteration  635
Number of training samples of Batch:  6155
Training loss:  0.5484638939512333
Begin iteration  636
Number of training samples of CSTR:  6384
Training loss

Training loss:  0.34659257742741006
Begin iteration  704
Number of training samples of CSTR:  6238
Training loss:  0.25895708979934084
Begin iteration  705
Number of training samples of Batch:  6240
Training loss:  0.42823500139964604
Begin iteration  706
Number of training samples of CSTR:  6400
Training loss:  0.3199987173827527
Begin iteration  707
Number of training samples of Batch:  6202
Training loss:  0.5140075702708806
Begin iteration  708
Number of training samples of CSTR:  6238
Training loss:  0.3596060101945166
Begin iteration  709
Number of training samples of Batch:  6240
Training loss:  0.3094204968213877
Begin iteration  710
Number of training samples of CSTR:  6400
Training loss:  0.2950345512767316
Begin iteration  711
Number of training samples of Batch:  5874
Training loss:  0.6997491336277875
Begin iteration  712
Number of training samples of CSTR:  6400
Training loss:  0.5127626763257507
Begin iteration  713
Number of training samples of Batch:  5880
Training los

Training loss:  0.3977079865420453
Begin iteration  784
Number of training samples of CSTR:  6400
Training loss:  0.2933838235280378
Begin iteration  785
Number of training samples of Batch:  6240
Training loss:  0.3111088075898194
Begin iteration  786
Number of training samples of CSTR:  6400
Training loss:  0.3264542311992914
Begin iteration  787
Number of training samples of Batch:  5930
Training loss:  0.624348031708941
Begin iteration  788
Number of training samples of CSTR:  6160
Training loss:  0.31748759340082927
Begin iteration  789
Number of training samples of Batch:  6240
Training loss:  0.2999230991933031
Begin iteration  790
Number of training samples of CSTR:  6367
Training loss:  0.2842639397612236
Begin iteration  791
Number of training samples of Batch:  6176
Training loss:  0.4689563862157384
Begin iteration  792
Number of training samples of CSTR:  6400
Training loss:  0.3797232618784125
Begin iteration  793
Number of training samples of Batch:  6240
Training loss: 

Number of training samples of Batch:  6235
Training loss:  0.43714345871733096
Begin iteration  862
Number of training samples of CSTR:  6160
Training loss:  0.28690755254073663
Begin iteration  863
Number of training samples of Batch:  6240
Training loss:  0.3200288372491792
Begin iteration  864
Number of training samples of CSTR:  6400
Training loss:  0.28435583134489373
Begin iteration  865
Number of training samples of Batch:  6240
Training loss:  0.4046502748832987
Begin iteration  866
Number of training samples of CSTR:  6160
Training loss:  0.3443736211279417
Begin iteration  867
Number of training samples of Batch:  6240
Training loss:  0.3530106413486037
Begin iteration  868
Number of training samples of CSTR:  6400
Training loss:  0.30592227205840705
Begin iteration  869
Number of training samples of Batch:  6240
Training loss:  0.3775576530351171
Begin iteration  870
Number of training samples of CSTR:  6400
Training loss:  0.29276418800549203
Begin iteration  871
Number of 

Number of training samples of Batch:  6240
Training loss:  0.27976451111929834
Begin iteration  942
Number of training samples of CSTR:  6329
Training loss:  0.3440176025962991
Begin iteration  943
Number of training samples of Batch:  6225
Training loss:  0.5241565939865146
Begin iteration  944
Number of training samples of CSTR:  6400
Training loss:  0.8215375443951979
Begin iteration  945
Number of training samples of Batch:  5980
Training loss:  0.6596609974809514
Begin iteration  946
Number of training samples of CSTR:  6286
Training loss:  0.3029950357384253
Begin iteration  947
Number of training samples of Batch:  6240
Training loss:  0.36763399868588104
Begin iteration  948
Number of training samples of CSTR:  6400
Training loss:  0.33302588292336666
Begin iteration  949
Number of training samples of Batch:  6240
Training loss:  0.28459655598229927
Test loss for 1-shot CSTR:  0.7241584571702001
Test loss for 5-shots CSTR:  0.3493987403240434
Test loss for 10-shots CSTR:  0.152

In [9]:
np.savetxt("training_reptile_cstr_batch.txt", training_loss, fmt='%f', delimiter=" ")
np.savetxt("testing_1shot_reptile_cstr_cstr_batch.txt", test_loss_1shot_cstr, fmt='%f', delimiter=" ")
np.savetxt("testing_5shot_reptile_cstr_cstr_batch.txt", test_loss_5shot_cstr, fmt='%f', delimiter=" ")
np.savetxt("testing_10shot_reptile_cstr_cstr_batch.txt", test_loss_10shot_cstr, fmt='%f', delimiter=" ")
np.savetxt("testing_1shot_reptile_batch_cstr_batch.txt", test_loss_1shot_batch, fmt='%f', delimiter=" ")
np.savetxt("testing_5shot_reptile_batch_cstr_batch.txt", test_loss_5shot_batch, fmt='%f', delimiter=" ")
np.savetxt("testing_10shot_reptile_batch_cstr_batch.txt", test_loss_10shot_batch, fmt='%f', delimiter=" ")

In [10]:
filename = 'model_reptile_cstr_batch.sav'
pickle.dump(model, open(filename, 'wb'))
# model = pickle.load(open(filename, 'rb'))